In [ ]:
!pip install gradio

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image

# Use FP16 to save memory, and load only ONE pipeline at a time
model_id = "lykon/dreamshaper-8"  # Smaller than SD 2.1 but effective

# Only load img2img or text2img when needed (don't load both at once)
def load_pipeline(task="img2img"):
    if task == "img2img":
        pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
            model_id,
            torch_dtype = torch.float16,
            variant="fp16",  # Use FP16 weights
            use_safetensors=True
        ).to("cuda")
    else:
        from diffusers import StableDiffusionPipeline
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True
        ).to("cuda")

    # Enable memory optimizations
    pipe.enable_attention_slicing()  # Reduces VRAM by slicing attention layers
    # pipe.enable_xformers_memory_efficient_attention()  # Optional but helps
    return pipe

def transform_image(image, prompt, strength=0.7):
    pipe = load_pipeline("img2img")
    result = pipe(
        prompt=prompt,
        image=image,
        strength=strength,
        num_inference_steps=50,
        guidance_scale=10
    ).images[0]

    # Cleanup to prevent memory leaks
    del pipe
    torch.cuda.empty_cache()
    return result

def generate_from_text(prompt):
    pipe = load_pipeline("text2img")
    result = pipe(
        prompt=prompt,
        height=768,
        width=768 ,
        guidance_scale=10 ,
        num_inference_steps=70
    ).images[0]

    del pipe
    torch.cuda.empty_cache()
    return result


In [ ]:
import gradio as gr
import torch
from PIL import Image, ImageEnhance


# Function to generate the image
def generate_image(modifications,user_image, body, custom_body, wings,custom_wings, color,custom_color, background,custom_background, appearance, mood, art_style,custom_art_style, quality, special_effects,custom_special_effects, manual_text, brightness, contrast, negative_prompt, mode):
    if mode == "Craft Your Own Visual Identity: A Custom Design Experience":
        # Resize the user's image to 512x512
        user_image = user_image.convert("RGB").resize((512, 512))

        image = transform_image(user_image, modifications)  # Pure img2img

        # Apply brightness and contrast adjustments
        image = ImageEnhance.Brightness(image).enhance(brightness)
        image = ImageEnhance.Contrast(image).enhance(contrast)

        return image
    else:
      # Use custom body type if selected
        if body == "Custom (Enter your own)":
            body = custom_body.strip() if custom_body else "Unknown Creature"

        if wings == "Custom (Enter your own)":
            wings = custom_wings.strip() if custom_wings else "Unknown Creature"

        if color == "Custom (Enter your own)":
            color = custom_color.strip() if custom_color else "Unknown Creature"

        if background == "Custom (Enter your own)":
            background = custom_background.strip() if custom_background else "Unknown Creature"

        if art_style == "Custom (Enter your own)":
            art_style = custom_art_style.strip() if custom_art_style else "Unknown art_style"

        if special_effects == "Custom (Enter your own)":
            special_effects = custom_special_effects.strip() if custom_special_effects else "Unknown special_effects"

        # Generate fantasy creature from scratch
        prompt = f"{body} with {wings} wings, {color} color, in {background}. {appearance}. Mood: {mood}. Art style: {art_style}. Quality: {quality}. Special effects: {special_effects}. {manual_text}"
        image = generate_from_text(prompt)  # Pure text2img
        return image

# UI Layout
with gr.Blocks() as demo:
    gr.Markdown(
        """
        # 🎨 MythicMorph :AI-Powered Creature Generator
        Design Your Own Legendary Creature: A Creative Exploration

        *Instructions:*
        - Choose to build from scratch or refine an existing fantasy concept into a personalized vision.
        - Customize a unique mythical being by selecting its traits, abilities, and appearance .
        """
    )

    # Mode selection
    mode = gr.Radio(
        choices=["Craft Your Own Visual Identity: A Custom Design Experience", "Forge a Myth: Craft Your Own Legendary Creature from Imagination"],
        label="Choose an option",
        value="Craft Your Own Visual Identity: A Custom Design Experience"
    )

    # Conditional UI for customizing own image
    with gr.Column(visible=True) as customize_image_ui:
        user_image = gr.Image(label="Upload Your Image", type="pil", )  # Default image
        modifications = gr.Textbox(label="Enter Modifications in Your Image", placeholder="E.g., Add horns, green skin, dragon wings")

    # Conditional UI for creating fantasy creature from scratch
    with gr.Column(visible=False) as fantasy_creature_ui:
        body_options = ["Custom (Enter your own)", "Same", "Dragon", "Phoenix", "Griffin", "Unicorn", "Wolf", "Mermaid", "Centaur", "Chimera", "Hydra", "Sphinx"]
        body = gr.Dropdown(body_options, label="Body Type", value="Dragon")
        custom_body = gr.Textbox(label="Enter your own body type", placeholder="e.g., Hybrid Tiger-Phoenix", visible=False)

        # Function to toggle visibility of custom body textbox
        def update_visibility(selected_body):
            return gr.update(visible=(selected_body == "Custom (Enter your own)"))

        body.change(update_visibility, inputs=body, outputs=custom_body)

        with gr.Row():
            wings = gr.Dropdown(["Feathered", "Bat-like", "None", "Mystical", "Angelic", "Custom (Enter your own)"], label="Wings", value="Feathered")
            custom_wings = gr.Textbox(label="Enter your own wings type", visible=False)
            def update_wings(selected_wings):
                return gr.update(visible=(selected_wings == "Custom (Enter your own)"))

            wings.change(update_wings, inputs=wings, outputs=custom_wings)

            color = gr.Dropdown(["Green", "Blue", "Fiery Red", "Purple", "Golden", "Custom (Enter your own)"], label="Color", value="Green")
            custom_color = gr.Textbox(label="Enter your own body type", visible=False)
            def update_color(selected_color):
                return gr.update(visible=(selected_color == "Custom (Enter your own)"))

            color.change(update_color, inputs=color, outputs=custom_color)

        with gr.Row():
            background = gr.Dropdown(["Custom (Enter your own)","Mystical Forest", "Scary Cave", "Sky Castle", "Magical Realm", "Custom"], label="Background", value="Mystical Forest")
            custom_background = gr.Textbox(label="Enter your own background type", visible=False)
            def update_background(selected_background):
                return gr.update(visible=(selected_background == "Custom (Enter your own)"))

            background.change(update_background, inputs=background, outputs=custom_background)

            appearance = gr.Textbox(label="Appearance Details", placeholder="E.g., glowing eyes, sharp fangs, intricate scales")
            mood = gr.Textbox(label="Mood & Atmosphere", placeholder="E.g., dark and mysterious, vibrant and cheerful")

        with gr.Row():
            art_style = gr.Dropdown(["Custom (Enter your own)","Realistic", "Anime", "Fantasy", "Cyberpunk", "Ghibli", "Surreal", "Gothic"], label="Art Style", value="Fantasy")
            custom_art_style = gr.Textbox(label="Enter your own art_style type", visible=False)
            def update_art_style(selected_art_style):
                return gr.update(visible=(selected_art_style == "Custom (Enter your own)"))

            art_style.change(update_art_style, inputs=art_style, outputs=custom_art_style)

            quality = gr.Dropdown(["Standard", "4k", "8k"], label="Quality", value="8k")

        with gr.Row():
            special_effects = gr.Dropdown(["Custom (Enter your own)","None", "Glowing", "Neon", "Shadowed", "Vintage", "Mystical Aura"], label="Special Effects", value="None")
            custom_special_effects = gr.Textbox(label="Enter your own special_effects type", visible=False)
            def update_special_effects(selected_special_effects):
                return gr.update(visible=(selected_art_style == "Custom (Enter your own)"))

            special_effects.change(update_special_effects, inputs=special_effects, outputs=custom_special_effects)

            manual_text = gr.Textbox(label="Additional Customization", placeholder="Add any extra details here...")

    # Common UI elements
    with gr.Row():
        brightness = gr.Slider(0.5, 2.0, value=1.0, label="Brightness")
        contrast = gr.Slider(0.5, 2.0, value=1.0, label="Contrast")


    with gr.Row():
        negative_prompt = gr.Textbox(label="Negative Prompt (optional)", placeholder="blurry, bad anatomy, distorted, deformed")

    output = gr.Image(label="Generated Image")
    generate_btn = gr.Button("✨ Generate Image")
    # Function to toggle UI based on mode
    def toggle_ui(mode):
        if mode == "Craft Your Own Visual Identity: A Custom Design Experience":
            return gr.update(visible=True), gr.update(visible=False)
        else:
            return gr.update(visible=False), gr.update(visible=True)

    mode.change(toggle_ui, inputs=mode, outputs=[customize_image_ui, fantasy_creature_ui])

    generate_btn.click(
        generate_image,
        inputs=[modifications,user_image, body, custom_body, wings,custom_wings, color,custom_color, background,custom_background, appearance, mood, art_style,custom_art_style, quality, special_effects,custom_special_effects ,manual_text, brightness, contrast, negative_prompt, mode],
        outputs=output
    )



# Launch the app
demo.launch(share=True)

In [ ]:
# Usage
if __name__ == "__main__":
    # user_image = Image.open("try_man2.jpeg").convert("RGB").resize((512, 512))
    prompt = '''Add horns , Big smile , Green Body , Dragon tatoo , long tooths , 8k'''
    # prompt = f"{body} with {wings} wings, {color} fur, in a {background}. {appearance} appearance, {mood} mood, {art_style} style, {quality} quality. {manual_text}"

    # Either use text2img OR img2img (don't chain them)
    final_image = generate_from_text(prompt)  # Pure img2img
    final_image.save("output.png")



from PIL import Image
Image.open("output.png")